# Examen 3° Parcial - Práctico
### Aitana Orduña Lugo

In [24]:
import pandas as pd
import os
from PIL import Image
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import load_model

In [2]:
path = r"C:\Users\holai\OneDrive\Escritorio\Semestre_6\MC2F\Neurociencias-2025-2\S03_datasets\Brain_tumor\Brain Tumor\Brain Tumor"
df = pd.read_csv(r"C:\Users\holai\OneDrive\Escritorio\Semestre_6\MC2F\Neurociencias-2025-2\S03_datasets\Brain_tumor\Brain Tumor.csv")

## 1. Aplicar CNN al datset y obtener el accuracy del modelo

In [3]:
# Arreglar .csv

exts = ['.jpg']

def find_image_with_extension(name):
    for ext in exts:
        full_path = os.path.join(path, name + ext)
        if os.path.exists(full_path):
            return name + ext
    return None

df["Image"] = df["Image"].apply(find_image_with_extension)

if df["Image"].isnull().any():
    print("Algunas imágenes no se encontraron")

df.to_csv(r"C:\Users\holai\OneDrive\Escritorio\Semestre_6\MC2F\Neurociencias-2025-2\S03_datasets\Brain_tumor\Brain Tumor actualizado.csv", index=False)

In [4]:
images = []
labels = []

IMG_SIZE = (150, 150)

for index, row in df.iterrows():
    img_path = os.path.join(path, row["Image"])
    if os.path.exists(img_path):
        img = Image.open(img_path).convert("RGB").resize(IMG_SIZE)
        images.append(np.array(img))
        labels.append(row["Class"])
    else:
        print(f"Imagen no encontrada: {img_path}")

In [5]:
X = np.array(images) / 255.0
y = to_categorical(labels)

In [6]:
# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print("Tamaño del dataset de entrenamiento:", X_train.shape)
print("Tamaño del dataset de prueba:", X_test.shape)

Tamaño del dataset de entrenamiento: (3009, 150, 150, 3)
Tamaño del dataset de prueba: (753, 150, 150, 3)


In [7]:
# Iniciando CNN 
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(2, activation='softmax')  
])

In [8]:
model.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.001),
    metrics=['accuracy']
)

In [9]:
history = model.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_test, y_test)
)

Epoch 1/10


95/95 [==============================] - 64s 504ms/step - loss: 0.4979 - accuracy: 0.7654 - val_loss: 0.3890 - val_accuracy: 0.8473
Epoch 2/10
95/95 [==============================] - 37s 394ms/step - loss: 0.3752 - accuracy: 0.8451 - val_loss: 0.2830 - val_accuracy: 0.8845
Epoch 3/10
95/95 [==============================] - 37s 388ms/step - loss: 0.3191 - accuracy: 0.8657 - val_loss: 0.2905 - val_accuracy: 0.8712
Epoch 4/10
95/95 [==============================] - 38s 402ms/step - loss: 0.2843 - accuracy: 0.8807 - val_loss: 0.2319 - val_accuracy: 0.9084
Epoch 5/10
95/95 [==============================] - 41s 435ms/step - loss: 0.2460 - accuracy: 0.9000 - val_loss: 0.2104 - val_accuracy: 0.9203
Epoch 6/10
95/95 [==============================] - 40s 425ms/step - loss: 0.2136 - accuracy: 0.9133 - val_loss: 0.1883 - val_accuracy: 0.9296
Epoch 7/10
95/95 [==============================] - 44s 459ms/step - loss: 0.1698 - accuracy: 0.9295 - val_loss: 0.2129 - val_accuracy: 0.92

In [10]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Accuracy en el conjunto de prueba: {test_acc:.4f}")

24/24 [==============================] - 3s 130ms/step - loss: 0.2275 - accuracy: 0.9296
Accuracy en el conjunto de prueba: 0.9296


In [22]:
model.save("modelo_cnn_1.keras")
print("Modelo guardado como modelo_cnn_1.keras")

Modelo guardado como modelo_cnn_1.keras


## 2. Mejorar el accuracy con # de capas, # de neuronas y funcion de activación

In [17]:
# Mejorando el Accuracy con otra CNN
## Hice algunos cambios nada más

model2 = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),

    Conv2D(64, (3,3)),
    LeakyReLU(alpha=0.1), #Agrego una capa

    Flatten(),
    Dense(256, activation='relu'), # Aumento el número de neuronas
    Dropout(0.5),
    Dense(2, activation='softmax')
])

In [18]:
model2.compile(
    loss='categorical_crossentropy',
    optimizer=Adam(learning_rate=0.0005), #Aprendizaje más lento pero más efectivo
    metrics=['accuracy']
)

In [19]:
history = model2.fit(
    X_train, y_train,
    epochs=10,
    batch_size=32,
    validation_data=(X_test, y_test)
)

Epoch 1/10
95/95 [==============================] - 58s 435ms/step - loss: 0.5086 - accuracy: 0.7660 - val_loss: 0.4545 - val_accuracy: 0.7689
Epoch 2/10
95/95 [==============================] - 36s 381ms/step - loss: 0.4246 - accuracy: 0.8172 - val_loss: 0.3169 - val_accuracy: 0.8632
Epoch 3/10
95/95 [==============================] - 35s 372ms/step - loss: 0.2939 - accuracy: 0.8754 - val_loss: 0.2605 - val_accuracy: 0.8884
Epoch 4/10
95/95 [==============================] - 36s 376ms/step - loss: 0.2579 - accuracy: 0.8983 - val_loss: 0.1964 - val_accuracy: 0.9283
Epoch 5/10
95/95 [==============================] - 36s 374ms/step - loss: 0.2110 - accuracy: 0.9179 - val_loss: 0.1954 - val_accuracy: 0.9336
Epoch 6/10
95/95 [==============================] - 35s 372ms/step - loss: 0.1749 - accuracy: 0.9335 - val_loss: 0.1430 - val_accuracy: 0.9469
Epoch 7/10
95/95 [==============================] - 36s 374ms/step - loss: 0.1378 - accuracy: 0.9505 - val_loss: 0.1283 - val_accuracy: 0.9509

In [20]:
loss, accuracy = model2.evaluate(X_test, y_test)
print(f"Accuracy en el conjunto de prueba: {accuracy:.4f}")

24/24 [==============================] - 3s 114ms/step - loss: 0.1748 - accuracy: 0.9416
Accuracy en el conjunto de prueba: 0.9416


In [23]:
model2.save("modelo_cnn_2.keras")
print("Modelo guardado como modelo_cnn_2.keras")

Modelo guardado como modelo_cnn_2.keras


In [26]:
# Comparando ambos modelos 

modelo_1 = load_model("modelo_cnn_1.keras")
modelo_2 = load_model("modelo_cnn_2.keras")

loss_1, acc_1 = modelo_1.evaluate(X_test, y_test, verbose=0)
loss_2, acc_2 = modelo_2.evaluate(X_test, y_test, verbose=0)

In [27]:
resultados = pd.DataFrame({
    "Modelo": ["CNN base", "CNN mejorado"],
    "Accuracy": [acc_1, acc_2]
})

print(resultados)

         Modelo  Accuracy
0      CNN base  0.929615
1  CNN mejorado  0.941567


## 3. Comparar los resultados de la CNN con los métodos previamente utilizados

In [47]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler

In [48]:
y_train_flat = np.argmax(y_train, axis=1)
y_test_flat = np.argmax(y_test, axis=1)

scaler = StandardScaler()
X_train_flat_scaled = scaler.fit_transform(X_train_flat)
X_test_flat_scaled = scaler.transform(X_test_flat)

In [36]:
# Random Forest
rf = RandomForestClassifier()
rf.fit(X_train_flat, y_train)
rf_acc = accuracy_score(y_test, rf.predict(X_test_flat))

In [49]:
# SVM
svm = SVC()
svm.fit(X_train_flat_scaled, y_train_flat)
svm_acc = accuracy_score(y_test_flat, svm.predict(X_test_flat_scaled))

In [38]:
# KNN
knn = KNeighborsClassifier()
knn.fit(X_train_flat, y_train)
knn_acc = accuracy_score(y_test, knn.predict(X_test_flat))

In [50]:
# Logistic Regression
lr = LogisticRegression(max_iter=1000)
lr.fit(X_train_flat_scaled, y_train_flat)
lr_acc = accuracy_score(y_test_flat, lr.predict(X_test_flat_scaled))

In [51]:
comparacion_modelos = pd.DataFrame({
    'Modelo': ['CNN Base', 'CNN Mejorado', 'Random Forest', 'SVM', 'KNN', 'Logistic Regression'],
    'Accuracy': [acc_1, acc_2, rf_acc, svm_acc, knn_acc, lr_acc]
})

print(comparacion_modelos)

                Modelo  Accuracy
0             CNN Base  0.929615
1         CNN Mejorado  0.941567
2        Random Forest  0.945551
3                  SVM  0.930943
4                  KNN  0.933599
5  Logistic Regression  0.884462
